In [5]:
import re
import asyncio
from playwright.async_api import async_playwright, Playwright

In [1]:
# # with open("cookies1.txt") as f:
# #     cookies = f.read()

# # import json

# # cookies = json.loads(cookies)

# import http.cookiejar

# cj = http.cookiejar.FileCookieJar("cookies_fb1.txt")
# print(cj)

In [2]:
import json

def first_json_by_path_finder(obj, key_str):
    keys = key_str.split(':')
    # print(keys)

    def find_json(to, k):
        if len(k) == 0:
            return to
        if isinstance(to, list):
            ret = []
            for i in to:
                r = find_json(i, k)
                if r is not None:
                    return r
            return None
        elif isinstance(to, dict):
            if k[0] == "**":
                if k[1] not in to:
                    for key in to.keys():
                        r = find_json(to[key], k)
                        if r is not None:
                            return r
                else:
                    return find_json(to[k[1]], k[2:])
            elif k[0] in to:
                return find_json(to[k[0]], k[1:])
            else:
                return None
        return None
    
    return find_json(obj, keys)

# randa pirma pasitaikiusi json objekta pagal nurodyta patha
# NAUDOJIMAS:

# 1 parametras, tai musu json objektas
# 2 parametras, tai kelias iki norimo objekto
#     formatas: **:comet_sections:**:profile_picture:uri
#     ** reiskia, kad tarp vieno ir kito objekto gali buti nuo 0 iki daugiau objektu


# first_json_by_path_finder(data ,'**:comet_sections:**:profile_picture:uri')

In [10]:
# SITA PALEISTI, TIK JEI BROWSERI RESTARTUOTI REIKIA

await browser.close()
await playwright.stop()

In [11]:
# await browser.close()
# await playwright.stop()

# CIA PAKEISKIT I savo duomenis, arba naudokit defaulta, kaip nurodyta playwhright dokumentacijoje

playwright = await async_playwright().start()
path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
user_data_dir = r"C:\Users\Tomas\AppData\Local\Google\Chrome\User Data\Tomas"
browser = await playwright.chromium.launch_persistent_context(headless=False, user_data_dir=user_data_dir, channel="chrome", executable_path=path)
page = await browser.new_page()
await page.goto('https://www.facebook.com/')

<Response url='https://www.facebook.com/' request=<Request url='https://www.facebook.com/' method='GET'>>

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


# PALEIDUS AUKSCIAU ESANTI FIELDA, PRISIJUNKITE PRIE FB, IR TIK TADA EIKIT TOLIAU






# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [12]:
# # neliesti
# cookies = await page.context.cookies()
# print(cookies)
# print(len(cookies))

In [18]:
def handle_field(field):
    post_data = {}
    post_data["user_id"] = first_json_by_path_finder(field, "**:actors:id")
    post_data["user_name"] = first_json_by_path_finder(field, "**:actors:name")
    post_data["profile_url"] = first_json_by_path_finder(field, "**:actors:url")

    post_data["post_id"] = first_json_by_path_finder(field, "**:story_ufi_container:story:post_id")
    post_data["post_url"] = first_json_by_path_finder(field, "**:story_ufi_container:story:url")
    # post_data["is_sponsored"] TODO
    post_data["post_creation_timestamp"] = first_json_by_path_finder(field, "**:metadata:story:creation_time")
    post_data["group_where_posted_id"] = first_json_by_path_finder(field, "**:target_group")
    if post_data["group_where_posted_id"] is not None:
        post_data["group_where_posted_id"] = post_data["group_where_posted_id"]["id"]

    post_data["reaction_count"] = first_json_by_path_finder(field, "**feedback:reaction_count:count")
    post_data["reactions"] = []
    reactions = first_json_by_path_finder(field, "**:top_reactions:edges")
    for reaction in reactions:
        post_data["reactions"].append({
            "reaction_id": reaction["node"]["id"],
            "reaction_name": reaction["node"]["localized_name"],
            "reaction_count": reaction["reaction_count"]
        })
    post_data["comment_count"] = first_json_by_path_finder(field, "**:comments:total_count")
    # TODO KOMENTARAI YRA KITAME REQUESTE!!!!!
    post_data["share_count"] = first_json_by_path_finder(field, "**:share_count:count")

    post_data["post_text"] = first_json_by_path_finder(field, "**:story:**:message:text")

    # prideti nuotrauku, video linkus, komentarus

    return post_data

In [15]:
def handle_fields(fields):
    cnt=0
    print("len fields",len(fields))
    for field in fields:
        print(cnt)
        cnt+=1
        f = json.loads(field)
        print(json.dumps(handle_field(f), indent=4))
        # hf = handle_field(f)
        # print(handle_field_for_commen(f))

In [16]:
async def print_request(request):
    if("https://www.facebook.com/api/graphql/" in request.url):
        try:
            text = await request.text()
            text = text.split('\n')
            fields = []
            for t in text:
                if "wwwURL" in t:
                    fields.append(t)
            print(">>", request.url, type(text), len(fields))
            if len(fields) > 3:
                handle_fields(fields)
        except Exception as e:
            print("except", request.url)
            print(e)

In [20]:
#!!!!!!!!!!!!!!

# SITA LEISTI 1 karta, pries paleidziant antra karta, paleiskit zemesni,
# kad pasalintumet sena
# todel nes duplikuojasi, ir kaskart bus tapati funkcija 2 kartus paleidziama

page.on("response", print_request)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

>> https://www.facebook.com/api/graphql/ <class 'list'> 0
>> https://www.facebook.com/api/graphql/ <class 'list'> 0
>> https://www.facebook.com/api/graphql/ <class 'list'> 5
len fields 5
0
{
    "user_id": "61550654891011",
    "user_name": "Knight-&-Aston",
    "profile_url": "https://www.facebook.com/profile.php?id=61550654891011",
    "post_id": "122193059978021829",
    "post_url": "https://www.facebook.com/permalink.php?story_fbid=pfbid0fo285WP1cWQxNxM7ymGtvoXn7qwnRNPNyJp77rPfV6VSXBknstPPyGuWrLnmG4BZl&id=61550654891011",
    "post_creation_timestamp": 1722303021,
    "group_where_posted_id": null,
    "reaction_count": null,
    "reactions": [
        {
            "reaction_id": "1635855486666999",
            "reaction_name": "Patinka",
            "reaction_count": 9
        },
        {
            "reaction_id": "478547315650144",
            "reaction_name": "Oho!",
            "reaction_count": 1
        }
    ],
    "comment_count": 2,
    "share_count": 2,
    "post_text"

In [19]:
# atjungti
page.remove_listener("response", print_request)